In [64]:
import pandas as pd
import numpy as np
import os, json
import nltk

In [65]:
DATAPATH = 'data'

In [66]:
csv_filepath ="/".join([DATAPATH, 'metadata.csv'])
df = pd.read_csv(csv_filepath)

In [67]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [68]:
df_sub = df[df['full_text_file'].notna() == True]
df_sub.shape

(42511, 18)

In [69]:
df_sub = df_sub[df_sub['abstract'].notna() == True]
df_sub.shape

(35074, 18)

In [70]:
df_sub = df_sub[df_sub['abstract'] != 'Unknown']
df_sub.shape

(34737, 18)

In [71]:
df_sub = df_sub[(df_sub['has_pdf_parse'] == True) | (df_sub['has_pmc_xml_parse'] == True)]
df_sub.shape

(33878, 18)

In [76]:
df_sub = df_sub[df['abstract'].str.len() >= 250]
df_sub.shape

C:\Users\Khachatur\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(33526, 18)

In [104]:
row = df_sub.iloc[[1]]
file_path = 'data/' + row.full_text_file[1] + '/' + row.full_text_file[1] + '/'

if row.has_pdf_parse[1]:
    file_path += 'pdf_json/' + row.sha[1] + '.json'
else:
    file_path += 'pmc_json/' + row.pmcid[1] + '.json'

with open(file_path) as json_file:
    data = json.load(json_file)
    print(json.dumps(data, indent=2))
    body_text = ''
    for section in data['body_text']:
        body_text += section['text'] + ' '
body_text

{
  "paper_id": "8ae137c8da1607b3a8e4c946c07ca8bda67f88ac",
  "metadata": {
    "title": "Discovering human history from stomach bacteria",
    "authors": [
      {
        "first": "Todd",
        "middle": [
          "R"
        ],
        "last": "Disotell",
        "suffix": "",
        "affiliation": {
          "laboratory": "",
          "institution": "New York University",
          "location": {
            "addrLine": "25 Waverly Place",
            "postCode": "10003",
            "settlement": "New York",
            "region": "NY",
            "country": "USA"
          }
        },
        "email": "todd.disotell@nyu.edu"
      }
    ]
  },
  "abstract": [
    {
      "text": "comment reviews reports deposited research interactions information refereed research Abstract Recent analyses of human pathogens have revealed that their evolutionary histories are congruent with the hypothesized pattern of ancient and modern human population migrations.",
      "cite_spans": [],

'Charles Darwin recognized that the distribution and form of parasites was evolutionarily significant. He noted, for instance, that "... the Pediculi [lice] collected in different countries from the different races of man ... differ, not only in colour, but in the structure of their claws and limbs. In every case in which many specimens were obtained the differences were constant" [1] . More recently, several research groups [2] [3] [4] [5] [6] [7] have found interesting correlations between the evolutionary relationships among various bacterial and viral strains hosted by humans and the pattern of migrations of modern humans throughout the world. A particularly interesting case is that of Helicobacter pylori, a Gram-negative bacterium associated with gastritis, peptic ulcers, and gastric cancer that may infect up to half of all humans [8] . The discovery that a bacterial infection could lead to what were considered chronic diseases [8] was a striking example of the fact that infectiou